## Import

In [120]:
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime, timezone, timedelta


# replace with the actual filenames
_diets_df = pd.read_csv("processed_final_diets.csv")
_ex_df    = pd.read_csv("processed_final_exercises.csv")


print(f"Loaded {len(_diets_df)} diets and {len(_ex_df)} exercises")


Loaded 8293 diets and 93 exercises


## Helper functions to simulate filtering out of data and exercises as well as pre-computations


In [121]:
def filter_diets(user_params: dict) -> pd.DataFrame:
    df = _diets_df.copy()
    dr = [d.lower() for d in user_params.get("dietRestrictions", [])]
    if "none" not in dr:
        df = df[df["diet_type"].str.lower().isin(dr)]
    vp = [v.lower() for v in user_params.get("varietyPreferences", [])]
    if "none" not in vp:
        df = df[df["cuisine_type"].str.lower().isin(vp)]
    print(f"Filtered diets ({len(df)} rows)")
    return df

def filter_exercises(user_params: dict) -> pd.DataFrame:
    df = _ex_df.copy()
    fl = user_params.get("fitnessLevel", "").lower()
    if fl and fl != "none":
        df = df[df["difficulty_level"].str.lower() == fl]
    pl = user_params.get("preferredLocation", "").lower()
    if pl and pl != "none":
        df = df[df["workout_location"].str.lower() == pl]
    wt = user_params.get("preferredWorkoutType", "").lower()
    if wt and wt != "none":
        df = df[df["activity_type"].str.lower() == wt]
    print(f"Filtered exercises ({len(df)} rows)")
    return df

In [122]:
from math import ceil


def compute_user_metrics(user: dict) -> dict:
    """Calculate user metrics based on personal information."""
    # Constants
    CALORIES_PER_KG = 7700  # Approximate calories per kg of body weight
    
    w, h, a = user["weight"], user["height"], user["age"]
    s = 5 if user["gender"].lower() == "male" else -161

    # 1. BMR
    bmr = 10 * w + 6.25 * h - 5 * a + s

    # 2. Activity multipliers (all lower-case keys)
    activity_map = {
        "Sedentary": 1.2,
        "Lightly Active": 1.375,
        "Moderately Active": 1.55,
        "Very Active": 1.725,
        "Extra Active": 1.9,
    }
    act = user["activityLevel"].lower()
    multiplier = activity_map.get(act, 1.2)
    tdee = bmr * multiplier

    # 3. Weight change & daily calorie delta
    gw = user["goalWeight"]
    wt_change = gw - w
    today = datetime.now(timezone.utc)
    tgt   = datetime.fromisoformat(user["goalTargetDate"].replace("Z","+00:00"))
    delta = tgt - today
    days = max( ceil(delta.total_seconds() / 86400), 1 )
    cal_delta = CALORIES_PER_KG * wt_change / days

    # 4. Target calories per day
    target_cal_pd = tdee + cal_delta

    return {
        "BMR": round(bmr,2),
        "TDEE": round(tdee,2),
        "weight_change_kg": round(wt_change,2),
        "days_to_target": days,
        "calorie_change_per_day": round(cal_delta,2),
        "target_calorie_per_day": round(target_cal_pd,2),
    }

## Example request body from frontend

In [130]:
sample_user = {
    "activityLevel": "Moderately Active",
    "age": 35,
    "daysWeek": 3,
    "dietRestrictions": ["none"],
    "fitnessLevel": "intermediate",
    "freeTime": 3,
    "gender": "male",
    "goalTargetDate": "2025-05-15T04:48:00.000Z",
    "goalType": "weight_loss",
    "goalWeight": 65,
    "height": 175,
    "mealPrepTime": 15,
    "mealsPerDay": 3,
    "name": "wei",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 70
}

## Prep work to calculate before feeding the details into Model

In [124]:
# filter data
diets_df = filter_diets(sample_user)
ex_df    = filter_exercises(sample_user)

# compute metrics
metrics = compute_user_metrics(sample_user)

# convert to lists of dicts for easy iteration in Gurobi
diet_options = diets_df.to_dict(orient="records")
ex_options   = ex_df.to_dict(orient="records")

print("User metrics:", metrics)
print(f"{len(diet_options)} diet options, {len(ex_options)} exercise options ready for modeling.")

Filtered diets (8293 rows)
Filtered exercises (57 rows)
User metrics: {'BMR': 1623.75, 'TDEE': 1948.5, 'weight_change_kg': -5, 'days_to_target': 32, 'calorie_change_per_day': -1203.12, 'target_calorie_per_day': 745.38}
8293 diet options, 57 exercise options ready for modeling.


In [125]:
print("Diet Dataset Preview:")
display(diets_df.head())
print(len(diets_df))
print("Exercise Dataset Preview:")
display(ex_df.head())
print(len(ex_df))



Diet Dataset Preview:


,recipe,diet_type,cuisine_type,total_time_in_minutes,calories,fat,carbs,protein
0,Chia And Blackberry Pudding,keto,kosher,45.0,437.0,38.0,8.0,8.0
1,Cinnamon Chiller,keto,south american,10.0,145.0,4.0,1.6,0.6
2,Cheesy Low-Carb Omelet,keto,chinese,10.0,451.0,36.0,3.0,33.0
3,Angel Eggs,keto,middle eastern,30.0,184.0,15.0,1.0,12.0
4,Roasted Onions And Green Beans,keto,italian,25.0,214.0,19.4,3.7,8.3


8293
Exercise Dataset Preview:


,exercise_name,calories_burned_30min,difficulty_level,calories_burned_per_min,workout_location,activity_type
0,Push-ups,200,Intermediate,6.666667,home,General
4,Mountain Climbers,240,Intermediate,8.000000,home,General
6,Bicycle Crunches,210,Intermediate,7.000000,home,General
7,Dips,180,Intermediate,6.000000,home,General
9,Russian Twists,190,Intermediate,6.333333,home,General


57


In [131]:
# More prep work
T = metrics["days_to_target"]
target_calorie_pd = metrics["target_calorie_per_day"]
goal = sample_user["goalType"].replace("_", " ")
# freeTime is in hours/day → convert to minutes/day
FT_day = sample_user["freeTime"] * 60
# mealPrepTimePerDay is total meal prep time (mins) per day
MPT_day = sample_user["mealPrepTime"]  * sample_user["mealsPerDay"]
# meals per day
M = sample_user["mealsPerDay"]
# desired workout days per week
W = sample_user["daysWeek"]

num_weeks = T // 7
has_partial_week = T % 7 > 0

print(f"num_days:{T}")
print(f"num_weeks: {num_weeks}, has_partial_week: {has_partial_week}")
print(f"Goal: {goal}")
print(f"Free time per day: {FT_day} mins")
print(f"Total meal‐prep time per day: {MPT_day} mins")
print(f"Meals per day: {M}")
print(f"Workout days per week: {W}")

num_days:32
num_weeks: 4, has_partial_week: True
Goal: weight loss
Free time per day: 180 mins
Total meal‐prep time per day: 45 mins
Meals per day: 3
Workout days per week: 3


## Model formulation and actual run-time

In [127]:
from gurobipy import Model, GRB, quicksum
model = Model("FitPlanner")
model.Params.OutputFlag = 0 # Turn off the display
model.setParam('TimeLimit', 300)  # Set time limit to 5 minutes

# Sets
days = range(T)
meals = list(diets_df.index)  # i in I
exercises = list(ex_df.index)  # j in J

# Parameters (extract from dataset)`
recipe = {i: diets_df.loc[i, 'recipe'] for i in meals}
cal = {i: diets_df.loc[i, 'calories'] for i in meals}
fat = {i: diets_df.loc[i, 'fat'] for i in meals}
carb = {i: diets_df.loc[i, 'carbs'] for i in meals}
protein = {i: diets_df.loc[i, 'protein'] for i in meals}
prep_time = {i: diets_df.loc[i, 'total_time_in_minutes'] for i in meals}
ex_name = {j: ex_df.loc[j, 'exercise_name'] for j in exercises}
location = {j: ex_df.loc[j, 'workout_location'] for j in exercises}
burn_rate = {j: ex_df.loc[j, 'calories_burned_per_min'] for j in exercises}
workout_type = {j: ex_df.loc[j, 'activity_type'] for j in exercises}

# Max time by workout type
max_time = {j: 60 if workout_type[j] in {"Sports"} else
            30 if workout_type[j] in {"Outdoor/Water"} else 20 for j in exercises}

# Variables
x = model.addVars(meals, days, vtype=GRB.BINARY, name="x")
y = model.addVars(exercises, days, vtype=GRB.BINARY, name="y")
t = model.addVars(exercises, days, vtype=GRB.CONTINUOUS, name="t")
s = model.addVars(days, vtype=GRB.BINARY, name="s")
Z = model.addVar(vtype=GRB.CONTINUOUS, name="Z")

# Objective: minimize deviation from target net calorie
model.setObjective(Z, GRB.MINIMIZE)

# Intake and burn
cal_intake = quicksum(x[i, d] * cal[i] for i in meals for d in days)
cal_burned = quicksum(t[j, d] * burn_rate[j] for j in exercises for d in days)

model.addConstr(Z >= cal_intake - cal_burned - T * target_calorie_pd)
model.addConstr(Z >= -(cal_intake - cal_burned - T * target_calorie_pd))

# Daily constraints
for d in days:
    intake_d = quicksum(x[i, d] * cal[i] for i in meals)
    burn_d = quicksum(t[j, d] * burn_rate[j] for j in exercises)
    prep_d = quicksum(x[i, d] * prep_time[i] for i in meals)
    exercise_d = quicksum(t[j, d] for j in exercises)

    # Calorie constraints based on goal
    if goal == "weight loss":
        model.addConstr(intake_d - burn_d <= target_calorie_pd)
    elif goal == "weight gain":
        model.addConstr(intake_d - burn_d >= target_calorie_pd)
    elif goal == "endurance":
        model.addConstr(intake_d - burn_d <= target_calorie_pd + 50)
        model.addConstr(intake_d - burn_d >= target_calorie_pd - 50)

    # Time constraints
    model.addConstr(prep_d <= MPT_day)
    model.addConstr(prep_d + exercise_d <= FT_day)

    # Meal count
    model.addConstr(quicksum(x[i, d] for i in meals) == M)

    # Link s[d] with exercise selection
    model.addConstr(s[d] <= quicksum(y[j, d] for j in exercises))
    model.addConstr(s[d] * 0.1 <= quicksum(y[j, d] for j in exercises))  # Added: If s[d]=1, at least one exercise
    
    for j in exercises:
        model.addConstr(t[j, d] <= y[j, d] * max_time[j])

    for i in meals:
        if d < T - 1:
            model.addConstr(x[i, d] + x[i, d + 1] <= 1)

    for j in exercises:
        if d < T - 1:
            model.addConstr(y[j, d] + y[j, d + 1] <= 1)

    # Macronutrient constraints
    fat_cal = quicksum(x[i, d] * fat[i] * 9 for i in meals)
    carb_cal = quicksum(x[i, d] * carb[i] * 4 for i in meals)
    protein_cal = quicksum(x[i, d] * protein[i] * 4 for i in meals)

    if goal in {"weight loss", "endurance"}:
        model.addConstr(fat_cal >= 0.2 * target_calorie_pd)
        model.addConstr(fat_cal <= 0.35 * target_calorie_pd)
        model.addConstr(carb_cal >= 0.45 * target_calorie_pd)
        model.addConstr(carb_cal <= 0.65 * target_calorie_pd)
        model.addConstr(protein_cal >= 0.1 * target_calorie_pd)
        model.addConstr(protein_cal <= 0.35 * target_calorie_pd)
    elif goal == "weight gain":
        model.addConstr(fat_cal >= 0.15 * target_calorie_pd)
        model.addConstr(fat_cal <= 0.3 * target_calorie_pd)
        model.addConstr(carb_cal >= 0.45 * target_calorie_pd)
        model.addConstr(carb_cal <= 0.6 * target_calorie_pd)
        model.addConstr(protein_cal >= 0.3 * target_calorie_pd)
        model.addConstr(protein_cal <= 0.35 * target_calorie_pd)

# Weekly workout constraint
for w in range(num_weeks):
    week_days = range(w * 7, (w + 1) * 7)
    model.addConstr(quicksum(s[d] for d in week_days) == W)

if has_partial_week:
    last_week_days = range(num_weeks * 7, T)
    model.addConstr(quicksum(s[d] for d in last_week_days) <= W)
    
# Optimize
model.optimize()

# Extract solution
if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
    print(f"Solution found with objective value: {model.objVal}")
    
    today = datetime.now()
    week_net_calories = 0
    week_total_time = 0
    workout_time_week = 0
    workout_days_count = 0

    for d in days:
        current_date = today + timedelta(days=d)
        meals_d = [i for i in meals if x[i, d].x > 0.5]
        
        # Only include exercises with positive duration (t > 0)
        exercises_d = [(j, t[j, d].x) for j in exercises if t[j, d].x > 0.001]  # Using small threshold to account for floating point precision

        intake = sum(cal[i] for i in meals_d)
        burned = sum(t[j, d].x * burn_rate[j] for j, _ in exercises_d)
        net = intake - burned
        meal_prep_time = sum(prep_time[i] for i in meals_d)
        exercise_time = sum(time for _, time in exercises_d)
        total_time = meal_prep_time + exercise_time
        day_date = current_date

        week_net_calories += net
        week_total_time += total_time
        workout_time_week += exercise_time
        if len(exercises_d) > 0:
            workout_days_count += 1

        if d % 7 == 0:
            print(f"\n=== Week {(d // 7) + 1} ===")

        print(f"Day {d + 1} ({day_date.strftime('%Y-%m-%d')}):")

        print(" Selected Meals:")
        for i in meals_d:
            print(f"    - {recipe[i]}, calories:{cal[i]} cal, carb: {carb[i]}g, protein: {protein[i]}g, fat: {fat[i]}g, total time: {prep_time[i]} mins")

        print(" Selected Exercises:")
        if exercises_d:
            for j, time in exercises_d:
                print(f"    - {ex_name[j]}, type: {workout_type[j]}, location: {location[j]}, duration: {time:.1f} mins, estimated calories burned: {burn_rate[j] * time:.1f} cal")
        else:
            print("    - No exercises scheduled for this day")

        print(f"  Total Time Spent: {total_time:.1f} mins")
        print(f"  Total Net Calories: {net:.1f} kcal")

    
elif model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Try relaxing some constraints.")
    # Compute and print the Irreducible Inconsistent Subsystem (IIS)
    model.computeIIS()
    print("The following constraints are causing the infeasibility:")
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"  {c.ConstrName}")
elif model.status == GRB.UNBOUNDED:
    print("Model is unbounded.")
else:
    print(f"Optimization ended with status {model.status}")

Solution found with objective value: 0.0

=== Week 1 ===
Day 1 (2025-04-13):
 Selected Meals:
    - The Ginger And Kale Ale, calories:140.0 cal, carb: 18.0g, protein: 1.0g, fat: 0.0g, total time: 5.0 mins
    - Tropical Storm Glass, calories:186.0 cal, carb: 14.0g, protein: 1.0g, fat: 0.0g, total time: 5.0 mins
    - Healthy & Delicious: Italian Egg-Drop Soup Recipe, calories:720.3675327719207 cal, carb: 84.14785911671517g, protein: 49.47829824683216g, fat: 20.65143370197016g, total time: 49.0 mins
 Selected Exercises:
    - Mountain Climbers, type: General, location: home, duration: 2.2 mins, estimated calories burned: 17.7 cal
    - Leg Raises, type: General, location: home, duration: 20.0 mins, estimated calories burned: 123.3 cal
    - Seated Rows, type: General, location: gym, duration: 20.0 mins, estimated calories burned: 160.0 cal
  Total Time Spent: 101.2 mins
  Total Net Calories: 745.4 kcal
Day 2 (2025-04-14):
 Selected Meals:
    - A Melon Cucumber Medley, calories:112.0 ca

## Diagnosis if model running failed

In [109]:
def diagnose_model(diets_df, ex_df, user_params, metrics):
    """
    Comprehensive diagnostic tool for the diet and exercise model.
    """
    print("\n=== MODEL DIAGNOSTICS ===\n")
    
    # 1. Check basic parameters
    T = metrics["days_to_target"]
    target_calorie_pd = metrics["target_calorie_per_day"]
    MPT_day = user_params["mealPrepTimePerDay"]
    FT_day = user_params["freeTime"] * 60
    M = user_params["mealsPerDay"]
    
    print(f"Planning horizon: {T} days")
    print(f"Target calories per day: {target_calorie_pd:.2f}")
    print(f"Meal prep time: {MPT_day} minutes")
    print(f"Free time: {FT_day} minutes")
    print(f"Meals per day: {M}")
    
    # 2. Check diet options
    print("\n--- Diet Options Analysis ---")
    print(f"Total diet options: {len(diets_df)}")
    
    # Time constraints
    avg_meal_time = diets_df['total_time_in_minutes'].mean()
    max_meal_time = diets_df['total_time_in_minutes'].max()
    min_meal_time = diets_df['total_time_in_minutes'].min()
    
    print(f"Average meal prep time: {avg_meal_time:.2f} minutes")
    print(f"Max meal prep time: {max_meal_time} minutes")
    print(f"Min meal prep time: {min_meal_time} minutes")
    
    # Check if meal prep time is feasible
    total_min_prep_time = min_meal_time * M
    if total_min_prep_time > MPT_day:
        print(f"⚠️ WARNING: Even the fastest {M} meals require {total_min_prep_time} minutes, but only {MPT_day} minutes available")
    
    # Calorie distribution
    avg_calories = diets_df['calories'].mean()
    max_calories = diets_df['calories'].max()
    min_calories = diets_df['calories'].min()
    
    print(f"Average meal calories: {avg_calories:.2f}")
    print(f"Max meal calories: {max_calories}")
    print(f"Min meal calories: {min_calories}")
    
    # Check if calorie target is feasible
    min_daily_calories = min_calories * M
    max_daily_calories = max_calories * M
    print(f"Min possible daily calories: {min_daily_calories}")
    print(f"Max possible daily calories: {max_daily_calories}")
    
    if target_calorie_pd < min_daily_calories:
        print(f"⚠️ WARNING: Target calories ({target_calorie_pd:.2f}) is less than minimum possible ({min_daily_calories})")
    if target_calorie_pd > max_daily_calories:
        print(f"⚠️ WARNING: Target calories ({target_calorie_pd:.2f}) is more than maximum possible ({max_daily_calories})")
    
    # Macronutrient analysis
    print("\n--- Macronutrient Analysis ---")
    
    # Calculate average macronutrient distribution
    avg_fat_cal = diets_df['fat'].mean() * 9
    avg_carb_cal = diets_df['carbs'].mean() * 4
    avg_protein_cal = diets_df['protein'].mean() * 4
    avg_total_cal = avg_fat_cal + avg_carb_cal + avg_protein_cal
    
    print(f"Average fat: {avg_fat_cal/avg_total_cal:.1%}")
    print(f"Average carbs: {avg_carb_cal/avg_total_cal:.1%}")
    print(f"Average protein: {avg_protein_cal/avg_total_cal:.1%}")
    
    # Check macronutrient constraints
    goal = user_params["goalType"]
    
    if goal in {"weight_loss", "endurance"}:
        fat_min, fat_max = 0.2, 0.35
        carb_min, carb_max = 0.45, 0.65
        protein_min, protein_max = 0.1, 0.35
    else:  # weight_gain
        fat_min, fat_max = 0.15, 0.3
        carb_min, carb_max = 0.45, 0.6
        protein_min, protein_max = 0.3, 0.35
    
    print(f"\nRequired macronutrient ranges for {goal}:")
    print(f"Fat: {fat_min:.1%} - {fat_max:.1%}")
    print(f"Carbs: {carb_min:.1%} - {carb_max:.1%}")
    print(f"Protein: {protein_min:.1%} - {protein_max:.1%}")
    
    # Check if any meals satisfy the macronutrient constraints
    meals_in_range = 0
    for _, meal in diets_df.iterrows():
        fat_cal = meal['fat'] * 9
        carb_cal = meal['carbs'] * 4
        protein_cal = meal['protein'] * 4
        total_cal = fat_cal + carb_cal + protein_cal
        
        fat_pct = fat_cal / total_cal
        carb_pct = carb_cal / total_cal
        protein_pct = protein_cal / total_cal
        
        if (fat_min <= fat_pct <= fat_max and 
            carb_min <= carb_pct <= carb_max and 
            protein_min <= protein_pct <= protein_max):
            meals_in_range += 1
    
    print(f"\nMeals satisfying macronutrient constraints: {meals_in_range} out of {len(diets_df)}")
    if meals_in_range < M:
        print(f"⚠️ WARNING: Not enough meals satisfy macronutrient constraints for {M} meals per day")
    
    # 3. Check exercise options
    print("\n--- Exercise Options Analysis ---")
    print(f"Total exercise options: {len(ex_df)}")
    
    # Calorie burn analysis
    avg_burn_rate = ex_df['calories_burned_per_min'].mean()
    max_burn_rate = ex_df['calories_burned_per_min'].max()
    min_burn_rate = ex_df['calories_burned_per_min'].min()
    
    print(f"Average burn rate: {avg_burn_rate:.2f} cal/min")
    print(f"Max burn rate: {max_burn_rate:.2f} cal/min")
    print(f"Min burn rate: {min_burn_rate:.2f} cal/min")
    
    # 4. Suggested relaxations
    print("\n--- Suggested Relaxations ---")
    
    if total_min_prep_time > MPT_day:
        print("1. Increase meal prep time or reduce meals per day")
    
    if target_calorie_pd < min_daily_calories or target_calorie_pd > max_daily_calories:
        print("2. Adjust target calories or add more diverse meal options")
    
    if meals_in_range < M:
        print("3. Relax macronutrient constraints or add more diverse meal options")
    
    print("4. Consider relaxing the food variation constraint (no same food on consecutive days)")
    print("5. Consider relaxing the exercise variation constraint")
    
    return {
        "meal_time_feasible": total_min_prep_time <= MPT_day,
        "calorie_target_feasible": min_daily_calories <= target_calorie_pd <= max_daily_calories,
        "macronutrient_feasible": meals_in_range >= M
    }

# Use this function before running the optimization
diagnostic_results = diagnose_model(diets_df, ex_df, sample_user, metrics)


=== MODEL DIAGNOSTICS ===

Planning horizon: 32 days
Target calories per day: 745.38
Meal prep time: 80 minutes
Free time: 300 minutes
Meals per day: 1

--- Diet Options Analysis ---
Total diet options: 8293
Average meal prep time: 39.85 minutes
Max meal prep time: 630.0 minutes
Min meal prep time: 0.0 minutes
Average meal calories: 619.85
Max meal calories: 880.0
Min meal calories: 0.24
Min possible daily calories: 0.24
Max possible daily calories: 880.0

--- Macronutrient Analysis ---
Average fat: 51.4%
Average carbs: 31.0%
Average protein: 17.5%

Required macronutrient ranges for weight_loss:
Fat: 20.0% - 35.0%
Carbs: 45.0% - 65.0%
Protein: 10.0% - 35.0%

Meals satisfying macronutrient constraints: 645 out of 8293

--- Exercise Options Analysis ---
Total exercise options: 57
Average burn rate: 7.70 cal/min
Max burn rate: 14.40 cal/min
Min burn rate: 2.33 cal/min

--- Suggested Relaxations ---
4. Consider relaxing the food variation constraint (no same food on consecutive days)
5. C

## Putting all of them together: Formatting the data into a format that Backend returns

In [128]:
from datetime import datetime, date, timedelta, timezone
from typing import List, Dict
from pydantic import BaseModel
from gurobipy import Model, GRB, quicksum

##############################
# Define Output Schema Models
##############################

class Meal(BaseModel):
    recipe: str
    calories: float
    macros: Dict[str, float]  # e.g. {"carbs": 55.0, "protein": 10.0, "fat": 8.0}
    total_time: float         # In minutes

class Exercise(BaseModel):
    name: str
    type: str
    location: str
    duration: float                # In minutes
    estimated_calories_burned: float

class DailyPlan(BaseModel):
    day: date
    selected_meals: List[Meal]
    selected_exercises: List[Exercise]
    total_time_used: float         # Combined time for meal prep and exercise (in minutes)
    total_net_calories: float      # Intake (from meals) minus calories burned (from exercise)

class WeeklyInfo(BaseModel):
    free_time_week: float             # Total free time available per week (in minutes)
    avg_free_time_used: float         # Average total time used per day (in minutes)
    avg_workout_duration: float       # Average workout duration per day (in minutes)
    meals_per_day: float              # Meals per day (as provided)
    avg_net_calories: float           # Average net calories per day

class OptimizationResult(BaseModel):
    plan: List[DailyPlan]
    weekly_info: WeeklyInfo
    status: str                     # The Gurobi model status (e.g., "OPTIMAL", "TIME_LIMIT", "INFEASIBLE")
    recommendations: List[str]      # List of recommendations to adjust backend inputs if needed

In [167]:
def diagnose_model(diets_df, ex_df, user_params, metrics) -> List[str]:
    """
    Aggregates smarter diagnostic recommendations for when the model is infeasible.
    Instead of listing all the computed numbers, it produces a summary
    and a concise recommendation of which input(s) might be changed.
    """
    recommendations = []

    T = metrics["days_to_target"]
    target_calorie_pd = metrics["target_calorie_per_day"]
    MPT_day = user_params.get("mealPrepTime") * user_params.get("mealsPerDay")
    FT_day = user_params["freeTime"] * 60
    M = user_params["mealsPerDay"]

    # Build a summary string of the key inputs:
    summary = (f"Planning horizon: {T} days; Target calories/day: {target_calorie_pd:.2f}; "
               f"Meal prep time: {MPT_day} min/day; Free time: {FT_day} min/day; Meals per day: {M}")

    # 1. Weight Change Feasibility:
    weight_change = user_params["goalWeight"] - user_params["weight"]
    if abs(weight_change) > 10 and T < 14:
        recommendations.append(f"Your target weight change ({weight_change} kg) is very aggressive for such a short period ({T} days). Consider reducing your weight change target or extending your planning horizon.")

    # 2. Meal Calorie Feasibility:
    min_calories = diets_df['calories'].min()
    max_calories = diets_df['calories'].max()
    min_daily_calories = min_calories * M
    max_daily_calories = max_calories * M
    # For weight loss, a low target calorie may be unachievable; for weight gain, a high target may be
    if target_calorie_pd < min_daily_calories:
        recommendations.append(f"Your planned calorie intake is too low for the current number of meals ({M} meals). Consider reducing the number of meals per day or extending your timeframe.")
    elif target_calorie_pd > max_daily_calories:
        recommendations.append(f"Your planned calorie intake is too high for the current number of meals ({M} meals). Consider increasing the number of meals per day or extending your timeframe.")

    # 3. Meal Prep Time Feasibility:
    min_meal_time = diets_df['total_time_in_minutes'].min()
    total_min_prep_time = min_meal_time * M
    if total_min_prep_time > MPT_day:
        recommendations.append("The fastest possible meal prep time exceeds your available time. Consider reducing meals per day or increasing your available meal prep time.")

    # 4. Macronutrient Feasibility:
    goal = user_params["goalType"]
    if goal in {"weight_loss", "endurance"}:
        fat_min, fat_max = 0.2, 0.35
        carb_min, carb_max = 0.45, 0.65
        protein_min, protein_max = 0.1, 0.35
    else:
        fat_min, fat_max = 0.15, 0.3
        carb_min, carb_max = 0.45, 0.6
        protein_min, protein_max = 0.3, 0.35

    meals_in_range = 0
    for _, meal in diets_df.iterrows():
        fat_cal = meal['fat'] * 9
        carb_cal = meal['carbs'] * 4
        protein_cal = meal['protein'] * 4
        total_cal = fat_cal + carb_cal + protein_cal
        fat_pct = fat_cal / total_cal
        carb_pct = carb_cal / total_cal
        protein_pct = protein_cal / total_cal
        if (fat_min <= fat_pct <= fat_max and
            carb_min <= carb_pct <= carb_max and
            protein_min <= protein_pct <= protein_max):
            meals_in_range += 1

    if meals_in_range < M:
        recommendations.append("There are not enough meals in our dataset that meet the desired macronutrient balance. Consider increasing the number of meals per day.")

    # 5. Exercise Options Feasibility:
    if len(ex_df) == 0:
        recommendations.append("No exercise options are available. Consider contacting support to add more exercise data.")

    # Combine the summary and the recommendations. (If no specific recommendation, just return the summary.)
    if recommendations:
        return [summary] + recommendations
    else:
        return [summary, "All input parameters appear feasible."]


In [161]:
GUROBI_STATUS_CODES = {
    1: "LOADED",
    2: "OPTIMAL",
    3: "INFEASIBLE",
    4: "INF_OR_UNBD",
    5: "UNBOUNDED",
    6: "CUTOFF",
    7: "ITERATION_LIMIT",
    8: "NODE_LIMIT",
    9: "TIME_LIMIT",
    10: "SOLUTION_LIMIT",
    11: "INTERRUPTED",
    12: "NUMERIC",
    13: "SUBOPTIMAL",
    14: "INPROGRESS",
    15: "USER_OBJ_LIMIT",
    16: "WORK_LIMIT",
    17: "MEM_LIMIT"
}

def generate_optimization_output(model: Model,
                                 days: range,
                                 meals: List,
                                 exercises: List,
                                 x, t,
                                 recipe: Dict,
                                 cal: Dict,
                                 fat: Dict,
                                 carb: Dict,
                                 protein: Dict,
                                 prep_time: Dict,
                                 ex_name: Dict,
                                 workout_type: Dict,
                                 location: Dict,
                                 burn_rate: Dict,
                                 user_params: Dict,
                                 metrics: Dict,
                                 diets_df, ex_df) -> dict:
    """
    Extracts the solution from the Gurobi model and aggregates the result in a JSON-compatible
    dictionary that conforms to the output schema.
    In addition to the optimization result (plan and weekly_info), it adds a 'status' field and
    a 'recommendations' field in case of failure.
    """
    # Start date for the plan is assumed to be today's date.
    start_date = datetime.now().date()
    plan: List[DailyPlan] = []
    recommendations: List[str] = []

    # Check model status: if solved (OPTIMAL or TIME_LIMIT) then extract solution.
    if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT]:
        total_time_week = 0.0
        total_exercise_time_week = 0.0
        total_net_cal_week = 0.0
        days_count_for_week = 0

        for d in days:
            current_date = start_date + timedelta(days=d)
            # Retrieve selected meals (using a threshold on x)
            selected_meal_indices = [i for i in meals if x[i, d].x > 0.5]
            # Retrieve exercises with positive duration.
            selected_exercise_indices = [(j, t[j, d].x) for j in exercises if t[j, d].x > 0.001]

            # Calculate daily totals.
            intake = sum(cal[i] for i in selected_meal_indices)
            burned = sum(t_val * burn_rate[j] for j, t_val in selected_exercise_indices)
            net = intake - burned
            meal_time_total = sum(prep_time[i] for i in selected_meal_indices)
            exercise_time_total = sum(t_val for _, t_val in selected_exercise_indices)
            total_time_used = meal_time_total + exercise_time_total

            # Construct Meal objects.
            selected_meals = []
            for i in selected_meal_indices:
                macros = {"carbs": float(carb[i]), "protein": float(protein[i]), "fat": float(fat[i])}
                meal_obj = Meal(
                    recipe=recipe[i],
                    calories=float(cal[i]),
                    macros=macros,
                    total_time=float(prep_time[i])
                )
                selected_meals.append(meal_obj)

            # Construct Exercise objects.
            selected_exercises = []
            for j, duration in selected_exercise_indices:
                ex_obj = Exercise(
                    name=ex_name[j],
                    type=workout_type[j],
                    location=location[j],
                    duration=duration,
                    estimated_calories_burned=duration * burn_rate[j]
                )
                selected_exercises.append(ex_obj)

            daily_plan = DailyPlan(
                day=current_date,
                selected_meals=selected_meals,
                selected_exercises=selected_exercises,
                total_time_used=total_time_used,
                total_net_calories=net
            )
            plan.append(daily_plan)

            if d < 7:  # For weekly aggregates, use the first 7 days.
                total_time_week += total_time_used
                total_exercise_time_week += exercise_time_total
                total_net_cal_week += net
                days_count_for_week += 1

        if days_count_for_week > 0:
            avg_free_time_used = total_time_week / days_count_for_week
            avg_workout_duration = total_exercise_time_week / days_count_for_week
            avg_net_calories = total_net_cal_week / days_count_for_week
        else:
            avg_free_time_used = avg_workout_duration = avg_net_calories = 0.0

        free_time_week = user_params.get("freeTime", 0) * 60 * user_params.get("daysWeek", 7)

        weekly_info = WeeklyInfo(
            free_time_week=free_time_week,
            avg_free_time_used=avg_free_time_used,
            avg_workout_duration=avg_workout_duration,
            meals_per_day=user_params.get("mealsPerDay", 0),
            avg_net_calories=avg_net_calories
        )
        status_str = GUROBI_STATUS_CODES[model.status]

    # If the model is infeasible, run diagnostics and gather recommendations.
    elif model.status == GRB.INFEASIBLE:
        recommendations = diagnose_model(diets_df, ex_df, user_params, metrics)
        status_str = GUROBI_STATUS_CODES[model.status]
        weekly_info = WeeklyInfo(
            free_time_week=0,
            avg_free_time_used=0,
            avg_workout_duration=0,
            meals_per_day=user_params.get("mealsPerDay", 0),
            avg_net_calories=0
        )
        plan = []  # No daily plans available.

    else:
        recommendations = [f"Model ended with status {model.status}."]
        status_str = GUROBI_STATUS_CODES[model.status]
        weekly_info = WeeklyInfo(
            free_time_week=0,
            avg_free_time_used=0,
            avg_workout_duration=0,
            meals_per_day=user_params.get("mealsPerDay", 0),
            avg_net_calories=0
        )
        plan = []

    # Assemble the final output dictionary.
    output = OptimizationResult(
        plan=plan,
        weekly_info=weekly_info,
        status=status_str,
        recommendations=recommendations
    )
    return output.model_dump()

## Optimal Case, No Recommendations to give to frontend

- Sufficient time to lose a target weight of 5kg with sufficient free time

In [162]:
result = generate_optimization_output(
    model=model,
    days=range(metrics["days_to_target"]),
    meals=list(diets_df.index),
    exercises=list(ex_df.index),
    x=x,
    t=t,
    recipe={i: diets_df.loc[i, 'recipe'] for i in diets_df.index},
    cal={i: diets_df.loc[i, 'calories'] for i in diets_df.index},
    fat={i: diets_df.loc[i, 'fat'] for i in diets_df.index},
    carb={i: diets_df.loc[i, 'carbs'] for i in diets_df.index},
    protein={i: diets_df.loc[i, 'protein'] for i in diets_df.index},
    prep_time={i: diets_df.loc[i, 'total_time_in_minutes'] for i in diets_df.index},
    ex_name={j: ex_df.loc[j, 'exercise_name'] for j in ex_df.index},
    workout_type={j: ex_df.loc[j, 'activity_type'] for j in ex_df.index},
    location={j: ex_df.loc[j, 'workout_location'] for j in ex_df.index},
    burn_rate={j: ex_df.loc[j, 'calories_burned_per_min'] for j in ex_df.index},
    user_params=sample_user,  # Your sample request body
    metrics=metrics,
    diets_df=diets_df,
    ex_df=ex_df
)

# Then print or return the JSON result to the frontend:
print(result)

{'plan': [{'day': datetime.date(2025, 4, 13), 'selected_meals': [{'recipe': 'The Ginger And Kale Ale', 'calories': 140.0, 'macros': {'carbs': 18.0, 'protein': 1.0, 'fat': 0.0}, 'total_time': 5.0}, {'recipe': 'Tropical Storm Glass', 'calories': 186.0, 'macros': {'carbs': 14.0, 'protein': 1.0, 'fat': 0.0}, 'total_time': 5.0}, {'recipe': 'Healthy & Delicious: Italian Egg-Drop Soup Recipe', 'calories': 720.3675327719207, 'macros': {'carbs': 84.14785911671517, 'protein': 49.47829824683216, 'fat': 20.65143370197016}, 'total_time': 49.0}], 'selected_exercises': [{'name': 'Mountain Climbers', 'type': 'General', 'location': 'home', 'duration': 2.2067749289900753, 'estimated_calories_burned': 17.654199431920603}, {'name': 'Leg Raises', 'type': 'General', 'location': 'home', 'duration': 20.0, 'estimated_calories_burned': 123.33333334000001}, {'name': 'Seated Rows', 'type': 'General', 'location': 'gym', 'duration': 20.0, 'estimated_calories_burned': 160.0}], 'total_time_used': 101.20677492899007, 

## Testing of more cases

### Helper function to make our lives easier to avoid dupication of code

In [163]:
def solve_optimization(user_params: dict) -> dict:
    """
    Runs the full optimization process given user parameters.
    It filters the data, computes metrics, builds and optimizes the model,
    and returns the results as a dictionary conforming to the output schema,
    along with a status and recommendations for infeasible cases.
    """
    # --- Preprocessing ---
    # Filter data based on user preferences.
    filtered_diets = filter_diets(user_params)
    filtered_exercises = filter_exercises(user_params)
    # Compute user metrics.
    metrics = compute_user_metrics(user_params)

    # Setup parameters for optimization.
    T = metrics["days_to_target"]
    target_calorie_pd = metrics["target_calorie_per_day"]
    goal = user_params["goalType"].replace("_", " ")
    FT_day = user_params["freeTime"] * 60
    MPT_day = user_params["mealPrepTime"] * user_params["mealsPerDay"]
    M = user_params["mealsPerDay"]
    W = user_params["daysWeek"]
    num_weeks = T // 7
    has_partial_week = T % 7 > 0

    print(f"num_days: {T}, num_weeks: {num_weeks}, has_partial_week: {has_partial_week}")
    print(f"Goal: {goal}")
    print(f"Free time per day: {FT_day} mins, Total meal prep time per day: {MPT_day} mins")
    print(f"Meals per day: {M}, Workout days per week: {W}")

    # --- Build Optimization Model ---
    model = Model("FitPlanner")
    model.Params.OutputFlag = 0
    model.setParam('TimeLimit', 300)  # 5 minutes time limit

    days = range(T)
    meals_idx = list(filtered_diets.index)
    exercises_idx = list(filtered_exercises.index)

    # Extract parameters from dataframes.
    recipe = {i: filtered_diets.loc[i, 'recipe'] for i in meals_idx}
    cal = {i: filtered_diets.loc[i, 'calories'] for i in meals_idx}
    fat = {i: filtered_diets.loc[i, 'fat'] for i in meals_idx}
    carb = {i: filtered_diets.loc[i, 'carbs'] for i in meals_idx}
    protein = {i: filtered_diets.loc[i, 'protein'] for i in meals_idx}
    prep_time = {i: filtered_diets.loc[i, 'total_time_in_minutes'] for i in meals_idx}
    ex_name = {j: filtered_exercises.loc[j, 'exercise_name'] for j in exercises_idx}
    location = {j: filtered_exercises.loc[j, 'workout_location'] for j in exercises_idx}
    burn_rate = {j: filtered_exercises.loc[j, 'calories_burned_per_min'] for j in exercises_idx}
    workout_type = {j: filtered_exercises.loc[j, 'activity_type'] for j in exercises_idx}

    max_time = {j: 60 if workout_type[j] in {"Sports"} else
                30 if workout_type[j] in {"Outdoor/Water"} else 20 for j in exercises_idx}

    # Decision variables.
    x = model.addVars(meals_idx, days, vtype=GRB.BINARY, name="x")
    y = model.addVars(exercises_idx, days, vtype=GRB.BINARY, name="y")
    t_var = model.addVars(exercises_idx, days, vtype=GRB.CONTINUOUS, name="t")
    s = model.addVars(days, vtype=GRB.BINARY, name="s")
    Z = model.addVar(vtype=GRB.CONTINUOUS, name="Z")

    model.setObjective(Z, GRB.MINIMIZE)
    cal_intake = quicksum(x[i, d] * cal[i] for i in meals_idx for d in days)
    cal_burned = quicksum(t_var[j, d] * burn_rate[j] for j in exercises_idx for d in days)
    model.addConstr(Z >= cal_intake - cal_burned - T * target_calorie_pd)
    model.addConstr(Z >= -(cal_intake - cal_burned - T * target_calorie_pd))

    for d in days:
        intake_d = quicksum(x[i, d] * cal[i] for i in meals_idx)
        burn_d = quicksum(t_var[j, d] * burn_rate[j] for j in exercises_idx)
        prep_d = quicksum(x[i, d] * prep_time[i] for i in meals_idx)
        exercise_d = quicksum(t_var[j, d] for j in exercises_idx)
        if goal == "weight loss":
            model.addConstr(intake_d - burn_d <= target_calorie_pd)
        elif goal == "weight gain":
            model.addConstr(intake_d - burn_d >= target_calorie_pd)
        elif goal == "endurance":
            model.addConstr(intake_d - burn_d <= target_calorie_pd + 50)
            model.addConstr(intake_d - burn_d >= target_calorie_pd - 50)
        model.addConstr(prep_d <= MPT_day)
        model.addConstr(prep_d + exercise_d <= FT_day)
        model.addConstr(quicksum(x[i, d] for i in meals_idx) == M)
        model.addConstr(s[d] <= quicksum(y[j, d] for j in exercises_idx))
        model.addConstr(s[d] * 0.1 <= quicksum(y[j, d] for j in exercises_idx))
        for j in exercises_idx:
            model.addConstr(t_var[j, d] <= y[j, d] * max_time[j])
        for i in meals_idx:
            if d < T - 1:
                model.addConstr(x[i, d] + x[i, d + 1] <= 1)
        for j in exercises_idx:
            if d < T - 1:
                model.addConstr(y[j, d] + y[j, d + 1] <= 1)
        fat_cal = quicksum(x[i, d] * fat[i] * 9 for i in meals_idx)
        carb_cal = quicksum(x[i, d] * carb[i] * 4 for i in meals_idx)
        protein_cal = quicksum(x[i, d] * protein[i] * 4 for i in meals_idx)
        if goal in {"weight loss", "endurance"}:
            model.addConstr(fat_cal >= 0.2 * target_calorie_pd)
            model.addConstr(fat_cal <= 0.35 * target_calorie_pd)
            model.addConstr(carb_cal >= 0.45 * target_calorie_pd)
            model.addConstr(carb_cal <= 0.65 * target_calorie_pd)
            model.addConstr(protein_cal >= 0.1 * target_calorie_pd)
            model.addConstr(protein_cal <= 0.35 * target_calorie_pd)
        elif goal == "weight gain":
            model.addConstr(fat_cal >= 0.15 * target_calorie_pd)
            model.addConstr(fat_cal <= 0.3 * target_calorie_pd)
            model.addConstr(carb_cal >= 0.45 * target_calorie_pd)
            model.addConstr(carb_cal <= 0.6 * target_calorie_pd)
            model.addConstr(protein_cal >= 0.3 * target_calorie_pd)
            model.addConstr(protein_cal <= 0.35 * target_calorie_pd)

    for w in range(num_weeks):
        week_days = range(w * 7, (w + 1) * 7)
        model.addConstr(quicksum(s[d] for d in week_days) == W)
    if has_partial_week:
        last_week_days = range(num_weeks * 7, T)
        model.addConstr(quicksum(s[d] for d in last_week_days) <= W)

    model.optimize()

    # Generate JSON-compatible output using the existing function.
    output = generate_optimization_output(
        model=model,
        days=days,
        meals=meals_idx,
        exercises=exercises_idx,
        x=x,
        t=t_var,
        recipe=recipe,
        cal=cal,
        fat=fat,
        carb=carb,
        protein=protein,
        prep_time=prep_time,
        ex_name=ex_name,
        workout_type=workout_type,
        location=location,
        burn_rate=burn_rate,
        user_params=user_params,
        metrics=metrics,
        diets_df=filtered_diets,
        ex_df=filtered_exercises
    )
    return output

In [164]:
# A helper to run a test case.
def test_case(case_name: str, user_params: dict):
    print(f"\n===== Test Case: {case_name} =====")
    result = solve_optimization(user_params)
    import json
    print(json.dumps(result, indent=2, default=str))

## Unrealistic weight loss

In [168]:
# Define some test scenarios:
# 1. Unrealistic weight loss: very high target weight change in too short period.
test_user_weight_loss = {
    "activityLevel": "Sedentary",
    "age": 30,
    "daysWeek": 3,
    "dietRestrictions": ["none"],
    "fitnessLevel": "beginner",
    "freeTime": 3,          # freeTime in hours per day
    "gender": "male",
    "goalTargetDate": "2025-05-15T04:48:00.000Z",  # Only a couple of days runway
    "goalType": "weight_loss",
    "goalWeight": 50,       # 20 kg weight loss
    "height": 175,
    "mealPrepTime": 15,
    "mealsPerDay": 3,
    "name": "TestUser1",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 70
}

# Run all test cases.
test_case("Unrealistic Weight Loss", test_user_weight_loss)


===== Test Case: Unrealistic Weight Loss =====
Filtered diets (8293 rows)
Filtered exercises (15 rows)
num_days: 32, num_weeks: 4, has_partial_week: True
Goal: weight loss
Free time per day: 180 mins, Total meal prep time per day: 45 mins
Meals per day: 3, Workout days per week: 3
{
  "plan": [],
  "weekly_info": {
    "free_time_week": 0.0,
    "avg_free_time_used": 0.0,
    "avg_workout_duration": 0.0,
    "meals_per_day": 3.0,
    "avg_net_calories": 0.0
  },
  "status": "INFEASIBLE",
  "recommendations": [
    "Planning horizon: 32 days; Target calories/day: -2834.00; Meal prep time: 45 min/day; Free time: 180 min/day; Meals per day: 3",
    "Your planned calorie intake is too low for the current number of meals (3 meals). Consider reducing the number of meals per day or extending your timeframe."
  ]
}


### Unrealistic Weight gain


In [169]:
# 2. Unrealistic weight gain: not enough time for the desired increase.
test_user_weight_gain = {
    "activityLevel": "Moderately Active",
    "age": 40,
    "daysWeek": 4,
    "dietRestrictions": ["none"],
    "fitnessLevel": "intermediate",
    "freeTime": 4,
    "gender": "female",
    "goalTargetDate": "2025-04-16T04:48:00.000Z",  # very short runway
    "goalType": "weight_gain",
    "goalWeight": 85,       # 20 kg gain
    "height": 165,
    "mealPrepTime": 20,
    "mealsPerDay": 4,
    "name": "TestUser2",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 65
}

test_case("Unrealistic Weight Gain", test_user_weight_gain)


===== Test Case: Unrealistic Weight Gain =====
Filtered diets (8293 rows)
Filtered exercises (57 rows)
num_days: 3, num_weeks: 0, has_partial_week: True
Goal: weight gain
Free time per day: 240 mins, Total meal prep time per day: 80 mins
Meals per day: 4, Workout days per week: 4
{
  "plan": [],
  "weekly_info": {
    "free_time_week": 0.0,
    "avg_free_time_used": 0.0,
    "avg_workout_duration": 0.0,
    "meals_per_day": 4.0,
    "avg_net_calories": 0.0
  },
  "status": "INFEASIBLE",
  "recommendations": [
    "Planning horizon: 3 days; Target calories/day: 52917.63; Meal prep time: 80 min/day; Free time: 240 min/day; Meals per day: 4",
    "Your target weight change (20 kg) is very aggressive for such a short period (3 days). Consider reducing your weight change target or extending your planning horizon.",
    "Your planned calorie intake is too high for the current number of meals (4 meals). Consider increasing the number of meals per day or extending your timeframe."
  ]
}


### Too little meals

In [170]:
# 3. Too few meals: eat 1 meal but want to gain weight
test_user_few_meals = {
    "activityLevel": "Lightly Active",
    "age": 28,
    "daysWeek": 3,
    "dietRestrictions": ["none"],
    "fitnessLevel": "beginner",
    "freeTime": 5,
    "gender": "female",
    "goalTargetDate": "2025-05-01T04:48:00.000Z",
    "goalType": "weight_gain",
    "goalWeight": 70,
    "height": 160,
    "mealPrepTime": 10,
    "mealsPerDay": 1,  # Requesting 1 meals per day (likely more than available)
    "name": "TestUser3",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 65
}

test_case("Too Few Meals Available", test_user_few_meals)


===== Test Case: Too Few Meals Available =====
Filtered diets (8293 rows)
Filtered exercises (15 rows)
num_days: 18, num_weeks: 2, has_partial_week: True
Goal: weight gain
Free time per day: 300 mins, Total meal prep time per day: 10 mins
Meals per day: 1, Workout days per week: 3
{
  "plan": [],
  "weekly_info": {
    "free_time_week": 0.0,
    "avg_free_time_used": 0.0,
    "avg_workout_duration": 0.0,
    "meals_per_day": 1.0,
    "avg_net_calories": 0.0
  },
  "status": "INFEASIBLE",
  "recommendations": [
    "Planning horizon: 18 days; Target calories/day: 3757.69; Meal prep time: 10 min/day; Free time: 300 min/day; Meals per day: 1",
    "Your planned calorie intake is too high for the current number of meals (1 meals). Consider increasing the number of meals per day or extending your timeframe."
  ]
}


### Too many meals requested

In [171]:
# 4. Too many meals: want to lose weight but also  eat a lot
test_user_many_meals = {
    "activityLevel": "Very Active",
    "age": 35,
    "daysWeek": 4,
    "dietRestrictions": ["none"],
    "fitnessLevel": "advanced",
    "freeTime": 2,
    "gender": "male",
    "goalTargetDate": "2025-05-01T04:48:00.000Z",
    "goalType": "weight_loss",
    "goalWeight": 65,
    "height": 180,
    "mealPrepTime": 10,
    "mealsPerDay": 7,  # Requesting 7 meals per day (might be too many if meal prep times add up)
    "name": "TestUser4",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 75
}

test_case("Too Many Meals Requested", test_user_many_meals)


===== Test Case: Too Many Meals Requested =====
Filtered diets (8293 rows)
Filtered exercises (21 rows)
num_days: 18, num_weeks: 2, has_partial_week: True
Goal: weight loss
Free time per day: 120 mins, Total meal prep time per day: 70 mins
Meals per day: 7, Workout days per week: 4
{
  "plan": [],
  "weekly_info": {
    "free_time_week": 0.0,
    "avg_free_time_used": 0.0,
    "avg_workout_duration": 0.0,
    "meals_per_day": 7.0,
    "avg_net_calories": 0.0
  },
  "status": "INFEASIBLE",
  "recommendations": [
    "Planning horizon: 18 days; Target calories/day: -2231.78; Meal prep time: 70 min/day; Free time: 120 min/day; Meals per day: 7",
    "Your planned calorie intake is too low for the current number of meals (7 meals). Consider reducing the number of meals per day or extending your timeframe."
  ]
}
